In [ ]:
import keras.layers as kl
import keras.optimizers as ko
import keras.preprocessing.image as kimg
from keras.applications.vgg16 import VGG16
from keras.models import Model

Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import io
import numpy as np
from os import makedirs
from os import listdir
from shutil import copyfile

In [ ]:
dataset='/content/drive/My Drive/train/dataset2/'

In [ ]:
label=['notes/','nonnotes/']

In [ ]:
for i in label:
  newdir=dataset+i
  makedirs(newdir,exist_ok=True)

In [ ]:
source='/content/drive/My Drive/train/train/'

In [ ]:
for file in listdir(source):
  src=source+'/'+file
  if file.startswith('notes'):
    dest=dataset+'notes/'+file
    copyfile(src,dest)
  elif file.startswith('non_notes'):
    dest=dataset+'nonnotes/'+file
    copyfile(src,dest)

In [ ]:
def modeldefinition():
  model=VGG16(include_top=False, input_shape=(224,224,3))
  for layers in model.layers:
    layers.trainable=False
  flat=kl.Flatten()(model.layers[-1].output)
  clas=kl.Dense(128,activation='relu',kernel_initializer='he_uniform')(flat)
  out=kl.Dense(1,activation='sigmoid')(clas)
  model=Model(inputs=model.inputs,outputs=out)
  opt=ko.SGD(lr=0.001,momentum=0.9)
  model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
  return model

In [ ]:
def testing():
  model=modeldefinition()
  data=kimg.ImageDataGenerator(featurewise_center=True)
  data.mean=[123.68,116.779,103.939]
  train_on=data.flow_from_directory('/content/drive/My Drive/train/dataset2/',class_mode='binary',batch_size=64, target_size=(224,224))
  model.fit_generator(train_on,steps_per_epoch=len(train_on),epochs=10,verbose=0)
  model.save('/content/drive/My Drive/train/vgg16model.h5')

In [ ]:
testing()

Found 1012 images belonging to 2 classes.


In [ ]:
def image(name):
  img=kimg.img_to_array(name)
  img=img.reshape(1,224,224,3)
  img=img.astype('float32')
  img=img-[123.68,116.779,103.939]
  return img

In [ ]:
from keras.models import load_model
from PIL import Image
import base64

In [ ]:
model=load_model('/content/drive/My Drive/train/vgg16model.h5')

In [ ]:
import tensorflow as tf

In [ ]:
model=tf.keras.models.load_model('/content/drive/My Drive/train/vgg16model.h5')

In [ ]:
converter=tf.lite.TFLiteConverter.from_keras_model(model)

In [ ]:
tflite_model = converter.convert()

In [ ]:
open('/content/drive/My Drive/train/converted_vgg16.tflite','wb').write(tflite_model)

71710920

In [ ]:
import pickle

In [ ]:
pickle.dump(model,open('/content/drive/My Drive/train/vgg16model.pkl','wb'))

In [ ]:
with open('/content/drive/My Drive/train/dataset_notes/test/notes/notes (226).jpg','rb') as file:
  encoded=base64.b64encode(file.read())

In [ ]:
decoded=base64.b64decode(encoded)

In [ ]:
img=Image.open(io.BytesIO(decoded))
img=img.resize((224,224))
img=image(img)
predictions=model.predict(img)
predictions[0][0]

1.0